In [ ]:
!pip install pandas openpyxl

In [ ]:
import pandas as pd
import smtplib
import time
import random
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import os

# --- CONFIGURATION ---
MY_EMAIL = "votre_email@gmail.com"
MY_PASSWORD = "xxxx xxxx xxxx xxxx"  # Votre mot de passe d'application (16 caractères)
CV_FILENAME = "mon_cv.pdf"           # Assurez-vous que ce fichier est dans le même dossier
EXCEL_FILENAME = "companies_extracted (1).xlsx" # Nom de votre fichier Excel

# Sujet de l'email
SUBJECT = "Candidature Spontanée: {role} - {name}"

# Modèle du corps de l'email (HTML autorisé)
# {contact_name} et {company_name} seront remplacés automatiquement
BODY_TEMPLATE = """
<html>
  <body>
    <p>Bonjour {contact_name},</p>
    
    <p>J'espère que ce courriel vous trouvera en bonne santé.</p>
    
    <p>Je me permets de vous contacter car je suis avec grand intérêt les activités de <strong>{company_name}</strong>.</p>
    
    <p>Actuellement à la recherche de nouvelles opportunités, je souhaite mettre mes compétences au service de votre équipe. Vous trouverez ci-joint mon CV détaillant mon parcours.</p>
    
    <p>Je serais ravi d'échanger avec vous lors d'un entretien.</p>
    
    <p>Cordialement,</p>
    <br>
    <p><strong>Votre Nom</strong><br>
    Votre Téléphone<br>
    <a href="https://linkedin.com/in/votreprofil">Mon Profil LinkedIn</a></p>
  </body>
</html>
"""

def send_emails():
    # 1. Chargement du fichier Excel
    try:
        # On utilise read_excel ici au lieu de read_csv
        df = pd.read_excel(EXCEL_FILENAME)
        print(f"Fichier chargé avec succès : {len(df)} lignes trouvées.")
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier Excel : {e}")
        return

    # 2. Connexion au serveur Gmail
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(MY_EMAIL, MY_PASSWORD)
        print("Connexion au serveur SMTP réussie.")
    except Exception as e:
        print(f"Échec de la connexion : {e}")
        return

    # 3. Boucle d'envoi
    count = 0
    
    for index, row in df.iterrows():
        # Récupération des colonnes (Assurez-vous que les noms correspondent exactement à votre Excel)
        email_dest = str(row.get('EntrepriseContactEmail', '')).strip()
        contact_name = str(row.get('EntrepriseContactName', '')).strip()
        company_name = str(row.get('EntrepriseName', '')).strip()

        # Nettoyage des valeurs 'nan' (not a number) que Pandas génère pour les cases vides
        if email_dest.lower() == 'nan' or email_dest == '':
            print(f"Ligne {index} ignorée : Pas d'email.")
            continue
            
        if contact_name.lower() == 'nan' or contact_name == '':
            contact_name = "l'équipe RH"
            
        if company_name.lower() == 'nan':
            company_name = "votre entreprise"

        # Personnalisation du message
        current_body = BODY_TEMPLATE.format(
            contact_name=contact_name, 
            company_name=company_name
        )

        # Création de l'email
        msg = MIMEMultipart()
        msg['From'] = MY_EMAIL
        msg['To'] = email_dest
        msg['Subject'] = SUBJECT.format(role="Développeur Full Stack", name="Votre Nom") # <-- Modifiez votre titre ici

        # Attachement du texte
        msg.attach(MIMEText(current_body, 'html'))

        # Attachement du PDF
        try:
            with open(CV_FILENAME, "rb") as f:
                attach = MIMEApplication(f.read(), _subtype="pdf")
                attach.add_header('Content-Disposition', 'attachment', filename=CV_FILENAME)
                msg.attach(attach)
        except FileNotFoundError:
            print(f"Erreur : Le fichier {CV_FILENAME} est introuvable. Arrêt du script.")
            break

        # Envoi
        try:
            server.sendmail(MY_EMAIL, email_dest, msg.as_string())
            count += 1
            print(f"[{count}] Envoyé à : {contact_name} chez {company_name} ({email_dest})")
        except Exception as e:
            print(f"Erreur d'envoi vers {email_dest}: {e}")

        # --- DÉLAI DE SÉCURITÉ ---
        # Pause aléatoire entre 60 et 120 secondes pour éviter le blocage Gmail
        sleep_time = random.randint(60, 120)
        print(f"Pause de {sleep_time} secondes...")
        time.sleep(sleep_time)

    server.quit()
    print("Terminé.")

if __name__ == "__main__":
    send_emails()